In [1]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value(tv).
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

In [2]:
from keras.utils.data_utils import get_file

path = "./data/"

filename_read = os.path.join(path,"Set For Deep-Learning Raw Data Water Storage Tank.csv")
df = pd.read_csv(filename_read)

print("Read {} rows.".format(len(df)))

# display 5 rows
df[0:5]

C:\Users\MuhammadAbdul\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Read 236179 rows.


,command_address,response_address,command_memory,response_memory,command_memory_count,response_memory_count,comm_read_function,comm_write_fun,resp_read_fun,resp_write_fun,...,H,L,LL,control_mode,control_scheme,pump,crc_rate,measurement,time,result
0,7,7,183,233,9,10,3,10,3,10,...,80,20,10,2,1,0,1,14.894827,1.13,0
1,7,7,183,233,9,10,3,10,3,10,...,80,20,10,2,1,0,1,16.259237,1.25,0
2,7,7,183,233,9,10,3,10,3,10,...,80,20,10,2,1,0,1,17.254122,1.04,0
3,7,7,183,233,9,10,3,10,3,10,...,80,20,10,2,1,0,1,17.936329,1.07,0
4,7,7,183,233,9,10,3,10,3,10,...,80,20,10,2,1,0,1,18.703810,1.20,0


In [3]:
ENCODING = 'utf-8'

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(filename):
    print()
    print("Analyzing: {}".format(filename))
    df = pd.read_csv(filename,encoding=ENCODING)
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

In [5]:
# Analyze Raw data set for water tank storage

import tensorflow.contrib.learn as skflow
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

path = "./data/"

filename_read = os.path.join(path,"Set For Deep-Learning Raw Data Water Storage Tank.csv")
analyze(filename_read)

In [7]:
# Now encode the feature vector

encode_numeric_zscore(df, 'command_address')
encode_numeric_zscore(df, 'response_address')
encode_numeric_zscore(df, 'command_memory') 
encode_numeric_zscore(df, 'response_memory')
encode_numeric_zscore(df, 'command_memory_count')
encode_numeric_zscore(df, 'response_memory_count')
encode_numeric_zscore(df, 'comm_read_function')
encode_numeric_zscore(df, 'comm_write_fun')
encode_numeric_zscore(df, 'resp_read_fun')
encode_numeric_zscore(df, 'resp_write_fun')
encode_numeric_zscore(df, 'sub_function')
encode_numeric_zscore(df, 'command_length')
encode_numeric_zscore(df, 'resp_length')
encode_numeric_zscore(df, 'HH')
encode_numeric_zscore(df, 'H')
encode_numeric_zscore(df, 'L')
encode_numeric_zscore(df, 'LL')
encode_numeric_zscore(df, 'control_mode')
encode_numeric_zscore(df, 'control_scheme')
encode_numeric_zscore(df, 'pump')
encode_numeric_zscore(df, 'crc_rate')
encode_numeric_zscore(df, 'measurement')
encode_numeric_zscore(df, 'time')
results = encode_text_index(df, 'result')
num_classes = len(results)

# display 5 rows

df.dropna(inplace=True,axis=1)
df[0:5]
# This is the numeric feature vector, as it goes to the neural net

,command_address,response_address,response_memory,command_memory_count,response_memory_count,comm_write_fun,resp_write_fun,sub_function,resp_length,HH,H,L,LL,control_mode,pump,crc_rate,measurement,time,result
0,-0.043227,0.410096,0.410096,-1.792576,0.410096,-0.428263,0.410096,-0.074969,-0.410096,0.138129,0.153081,0.082842,-0.094258,0.663985,-0.081515,1.792576,-0.000955,0.009757,0
1,-0.043227,0.410096,0.410096,-1.792576,0.410096,-0.428263,0.410096,-0.074969,-0.410096,0.138129,0.153081,0.082842,-0.094258,0.663985,-0.081515,1.792576,-0.000955,1.271971,0
2,-0.043227,0.410096,0.410096,-1.792576,0.410096,-0.428263,0.410096,-0.074969,-0.410096,0.138129,0.153081,0.082842,-0.094258,0.663985,-0.081515,1.792576,-0.000955,-0.936903,0
3,-0.043227,0.410096,0.410096,-1.792576,0.410096,-0.428263,0.410096,-0.074969,-0.410096,0.138129,0.153081,0.082842,-0.094258,0.663985,-0.081515,1.792576,-0.000955,-0.621349,0
4,-0.043227,0.410096,0.410096,-1.792576,0.410096,-0.428263,0.410096,-0.074969,-0.410096,0.138129,0.153081,0.082842,-0.094258,0.663985,-0.081515,1.792576,-0.000955,0.746049,0


In [8]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping

# Break into X (predictors) & y (prediction)
x, y = to_xy(df,'result')

# Create a test/train split.  25% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Train on 177134 samples, validate on 59045 samples
Epoch 1/1000
 - 15s - loss: 0.5287 - val_loss: 0.4017
Epoch 2/1000
 - 8s - loss: 0.3825 - val_loss: 0.3632
Epoch 3/1000
 - 8s - loss: 0.3578 - val_loss: 0.3489
Epoch 4/1000
 - 8s - loss: 0.3458 - val_loss: 0.3364
Epoch 5/1000
 - 8s - loss: 0.3399 - val_loss: 0.3315
Epoch 6/1000
 - 7s - loss: 0.3361 - val_loss: 0.3269
Epoch 7/1000
 - 7s - loss: 0.3329 - val_loss: 0.3248
Epoch 8/1000
 - 7s - loss: 0.3300 - val_loss: 0.3207
Epoch 9/1000
 - 8s - loss: 0.3267 - val_loss: 0.3330
Epoch 10/1000
 - 7s - loss: 0.3247 - val_loss: 0.3180
Epoch 11/1000
 - 8s - loss: 0.3216 - val_loss: 0.3152
Epoch 12/1000
 - 8s - loss: 0.3200 - val_loss: 0.3108
Epoch 13/1000
 - 7s - loss: 0.3176 - val_loss: 0.3097
Epoch 14/1000
 - 7s - loss: 0.3172 - val_loss: 0.3113
Epoch 15/1000
 - 8s - loss: 0.3168 - val_loss: 0.3089
Epoch 16/1000
 - 7s - loss: 0.3164 - val_loss: 0.3189
Epoch 17/1000
 - 8s - loss: 0.3159 - val_loss: 0.3086
Epoch 18/1000
 - 8s - loss: 0.3156 - va

In [9]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

Validation score: 0.9089508002371073
